In [1]:
import glob
from elasticsearch import Elasticsearch

# Zadania 3 i 4

stworzenie indexu z customowym analyzerem zawierającym słownik synonimów, polski lematyzator i filtr lowercase

In [2]:
es = Elasticsearch()

In [3]:
es.indices.create(
    index="custom_index",
    body={
        "settings": {
            "analysis": {
                "filter": {
                    "custom_synonym": {
                        "type": "synonym",
                        "synonyms": [ 
                            "kpk => kodeks postępowania karnego",
                            "kpc => kodeks postępowania cywilnego",
                            "kk => kodeks karny",
                            "kc => kodeks cywilny"
                        ]
                    }    
                },
                "analyzer": {
                  "custom_analyzer": {
                      "type": "custom",
                      "tokenizer": "standard",
                      "filter": ["custom_synonym", "morfologik_stem", "lowercase"]
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "law_act_content": {
                    "type": "text",
                    "analyzer": "custom_analyzer"
                }
            }
        }
    }
)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'custom_index'}

In [4]:
# es.indices.delete(index="custom_index")

# Zadanie 5

wczytanie plików projektów ustaw do indexu

In [5]:
PATH = "../lab1/ustawy/"
file_names = sorted(glob.glob(PATH + "*"))
file_names[:5]

['../lab1/ustawy/1993_599.txt',
 '../lab1/ustawy/1993_602.txt',
 '../lab1/ustawy/1993_645.txt',
 '../lab1/ustawy/1993_646.txt',
 '../lab1/ustawy/1994_150.txt']

In [6]:
for file_name in file_names:
    with open(file_name, "r", encoding="utf-8") as f:
        law_act = f.read()
        es.create("custom_index", file_name.split("/")[-1][:-4], {"law_act_content": law_act})

# Zadanie 6

Znalezienie liczby wystąpień słowa ustawa w dowolnej formie

Najprostszy przypadek - zastosowanie zwykłego match query

In [7]:
es.search(
    body={
        "query": {
            "match": {
                "law_act_content": "ustawa"
            }
        }
    },
    index="custom_index"
)["hits"]["total"]["value"]

1179

# Zadanie 7

Znalezienie liczby projektów ustaw zawierających frazę "*kodeks postępowania cywilnego*" w dowolnej formie, ale ustalonej kolejności

Warunek zachowania kolejności bez warunku niezmiennej formy zapewnia wykorzystanie match_phrase query z parametrem slop ustawionym domyślnie na 0

In [8]:
es.search(
    body={
        "query": {
            "match_phrase": {
                "law_act_content": "kodeks postępowania cywilnego"
            }
        }
    },
    index="custom_index"
)["hits"]["total"]["value"]

100

# Zadanie 8

znalezienie liczby projektów ustaw zawierających słowa "*wchodzi w życie*" w dowolnej formie, z maksymalnie 2 dodatkowymi słowami

Możliwość pojawienia się 2 dowolnych słów w środku frazy zapewnia użycie match_phrase z parametrem slop = 2

In [9]:
es.search(
    body={
        "query": {
            "match_phrase": {
                "law_act_content": {
                    "query": "wchodzi w życie",
                    "slop": 2
                }
            }
        }
    },
    index="custom_index"
)["hits"]["total"]["value"]

1175

# Zadanie 9 i 10

znalezienie 10 dokumentów, które są najbardziej związane ze słowem konstytucja i wypisywanie ich fragmentów zawierających to słowo (max 3)

Aby poznać scoring matchów wystarczy użyć zwykłego match query. Dodatkowo aby mieć dostęp do fragmentów dokumentów zawierających to słowo stosuję mechanizm highlightingu, który domyślnie zachowuje zdania, w których zmatchowano słowo konstytucja. Ustawienie parametru highlightera "order" na "score" powoduje, że otrzymane fragmenty będą posortowane od tych które najbardziej podbijały score, a ustawienie "number_of_fragments" na 3 wymusza zachowanie max 3 fragmentów. Parametr size ustawiony na 10 wymusza zwrot dokładnie 10 najlepszych matchy.

In [10]:
results = es.search(
    body={
        "query": {
            "match": {
                "law_act_content": "konstytucja"
            }
        },
        "highlight": {
            "fields": {
                "law_act_content": {}
            },
            "order": "score",
            "number_of_fragments": 3
        },
    },
    index="custom_index", 
    size=10
)

for document in results["hits"]["hits"]:
    print(document["_id"], "with score", document["_score"])

1997_629 with score 6.8713923
2000_443 with score 6.666975
1997_604 with score 6.6366324
1996_350 with score 6.6311083
1997_642 with score 6.2556105
2001_23 with score 6.0630813
1996_199 with score 5.9332457
1999_688 with score 5.854491
1997_681 with score 5.4713526
2001_1082 with score 5.4713526


In [11]:
for document in results["hits"]["hits"]:
    print('\033[1m', document["_id"], "\033[0m:\n")
    for fragment in document["highlight"]["law_act_content"]:
        print(fragment, "\n------------------------------------------------\n")


 1997_629 :

W ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i 
uchwalenia <em>Konstytucji</em> 
------------------------------------------------

o zmianie ustawy konstytucyjnej o trybie przygotowania
           i uchwalenia <em>Konstytucji</em> Rzeczypospolitej 
------------------------------------------------

Do zgłoszenia projektu <em>Konstytucji</em> załącza się wykaz 
                obywateli popierających zgłoszenie 
------------------------------------------------

 2000_443 :

umowy międzynarodowej i nie wypełnia przesłanek określonych w art. 89
     ust. 1 lub art. 90 <em>Konstytucji</em> 
------------------------------------------------

co do zasadności wyboru
  trybu ratyfikacji umowy międzynarodowej, o którym mowa w art. 89 ust. 2
  <em>Konstytucji</em> 
------------------------------------------------

międzynarodowej lub załącznika nie
     wypełnia przesłanek określonych w art. 89 ust. 1 lub art. 90 <em>Konstytucji</em> 
-----------------